In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt       # Standard plotting library


# Pretty display for notebooks
%matplotlib inline

# Load the Boston housing dataset
X = pd.read_csv('data/application_train_final.csv')#application_train_final.csv')
# Success - Display the first record
display(X.head(n=5))


,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,CNT_CHILDREN,CNT_FAM_MEMBERS,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,...,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY
0,0.537494,0.489166,0.468924,0.245232,0.0,0.00,0.881337,0.035563,0.705433,0.739968,...,0,0,0,0,0,0,0,0,0,1
1,0.610092,0.746352,0.722714,0.279376,0.0,0.05,0.473087,0.066324,0.959566,0.810135,...,0,0,0,0,1,0,0,0,0,0
2,0.281783,0.244144,0.261437,0.114839,0.0,0.00,0.345593,0.012561,0.648326,0.813116,...,0,0,0,0,1,0,0,0,0,0
3,0.573739,0.430800,0.432648,0.197108,0.0,0.05,0.347884,0.169663,0.661387,0.858519,...,0,0,0,0,0,0,0,0,0,1
4,0.513463,0.540824,0.551329,0.184602,0.0,0.00,0.296071,0.169607,0.519522,0.746388,...,0,0,0,0,0,0,0,1,0,0


In [2]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

X.dropna(subset=['TARGET'], inplace=True)

target = 'TARGET'
IDcol = 'SK_ID_CURR'

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : {}".format(metrics.accuracy_score(dtrain[target].values, dtrain_predictions)))
    print ("AUC Score (Train): {}".format(metrics.roc_auc_score(dtrain[target], dtrain_predprob)))
    print ("Precision (Train): {}".format(metrics.precision_score(dtrain[target].values, dtrain_predictions)))
    print ("Recall (Train): {}".format(metrics.recall_score(dtrain[target].values, dtrain_predictions)))
    tn, fp, fn, tp = metrics.confusion_matrix(dtrain[target].values, dtrain_predictions).ravel()
    print("TN: {}, FP: {}, FN: {}, TP: {}".format(tn, fp, fn, tp))
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [4]:
#Choose all predictors except target & IDcols
predictors = [x for x in X.columns if x not in [target, IDcol]]
#xgb1 = XGBClassifier(
# learning_rate =0.1,
# n_estimators=1000,
# max_depth=5,
# min_child_weight=1,
# gamma=0,
# subsample=0.8,
# colsample_bytree=0.8,
# objective= 'binary:logistic',
# nthread=4,
# scale_pos_weight=1,
# seed=27)
#modelfit(xgb1, X[:20000], predictors)

# GridSearch for max depth and min_child_weight

In [11]:
import datetime
print(datetime.datetime.now())

param_test1 = {
 'max_depth':[4,6,8,10],
 'min_child_weight':[2,4,6,8]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X[predictors],X[target])
print(datetime.datetime.now())
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

2018-10-16 07:00:32.139573
2018-10-16 12:12:20.341606


([mean: 0.71108, std: 0.00244, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.71088, std: 0.00309, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.71058, std: 0.00301, params: {'max_depth': 4, 'min_child_weight': 6},
  mean: 0.71097, std: 0.00331, params: {'max_depth': 4, 'min_child_weight': 8},
  mean: 0.71549, std: 0.00257, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.71557, std: 0.00254, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.71620, std: 0.00307, params: {'max_depth': 6, 'min_child_weight': 6},
  mean: 0.71520, std: 0.00227, params: {'max_depth': 6, 'min_child_weight': 8},
  mean: 0.71313, std: 0.00345, params: {'max_depth': 8, 'min_child_weight': 2},
  mean: 0.71324, std: 0.00278, params: {'max_depth': 8, 'min_child_weight': 4},
  mean: 0.71443, std: 0.00269, params: {'max_depth': 8, 'min_child_weight': 6},
  mean: 0.71486, std: 0.00269, params: {'max_depth': 8, 'min_child_weight': 8},
  mean: 0.70688, std: 0.00283, params: {

In [11]:
import datetime
print(datetime.datetime.now())

param_test1 = {
 'max_depth':[4,6,8,10],
 'min_child_weight':[2,4,6,8]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X[predictors],X[target])
print(datetime.datetime.now())
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

2018-10-16 07:00:32.139573
2018-10-16 12:12:20.341606


([mean: 0.71108, std: 0.00244, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.71088, std: 0.00309, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.71058, std: 0.00301, params: {'max_depth': 4, 'min_child_weight': 6},
  mean: 0.71097, std: 0.00331, params: {'max_depth': 4, 'min_child_weight': 8},
  mean: 0.71549, std: 0.00257, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.71557, std: 0.00254, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.71620, std: 0.00307, params: {'max_depth': 6, 'min_child_weight': 6},
  mean: 0.71520, std: 0.00227, params: {'max_depth': 6, 'min_child_weight': 8},
  mean: 0.71313, std: 0.00345, params: {'max_depth': 8, 'min_child_weight': 2},
  mean: 0.71324, std: 0.00278, params: {'max_depth': 8, 'min_child_weight': 4},
  mean: 0.71443, std: 0.00269, params: {'max_depth': 8, 'min_child_weight': 6},
  mean: 0.71486, std: 0.00269, params: {'max_depth': 8, 'min_child_weight': 8},
  mean: 0.70688, std: 0.00283, params: {

In [7]:
param_test1 = {
 'max_depth':[1,2,3],
 'min_child_weight':[1,2,3]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X[predictors],X[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will res

([mean: 0.00000, std: 0.00000, params: {'max_depth': 1, 'min_child_weight': 1},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 1, 'min_child_weight': 2},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 1, 'min_child_weight': 3},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 2, 'min_child_weight': 1},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 2, 'min_child_weight': 2},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 2, 'min_child_weight': 3},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 3, 'min_child_weight': 2},
  mean: 0.00000, std: 0.00000, params: {'max_depth': 3, 'min_child_weight': 3}],
 {'max_depth': 1, 'min_child_weight': 1},
 0.0)

In [9]:
param_test1 = {
 'max_depth':[4, 5, 6, 7],
 'min_child_weight':[1, 2, 3, 4]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X[predictors][:20000],X[target][:20000])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will res

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will res

([mean: 0.00000, std: 0.00000, params: {'max_depth': 4, 'min_child_weight': 1},
  mean: 0.00252, std: 0.00309, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.00126, std: 0.00252, params: {'max_depth': 4, 'min_child_weight': 3},
  mean: 0.00126, std: 0.00252, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.00126, std: 0.00252, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.00126, std: 0.00252, params: {'max_depth': 5, 'min_child_weight': 2},
  mean: 0.00253, std: 0.00310, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.00126, std: 0.00252, params: {'max_depth': 5, 'min_child_weight': 4},
  mean: 0.00377, std: 0.00502, params: {'max_depth': 6, 'min_child_weight': 1},
  mean: 0.00877, std: 0.00847, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.00252, std: 0.00309, params: {'max_depth': 6, 'min_child_weight': 3},
  mean: 0.00252, std: 0.00309, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.00748, std: 0.00464, params: {

In [10]:
param_test1 = {
 'max_depth':[6, 8, 9, 10],
 'min_child_weight':[1, 2, 3, 4]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X[predictors][:20000],X[target][:20000])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

([mean: 0.00377, std: 0.00502, params: {'max_depth': 6, 'min_child_weight': 1},
  mean: 0.00877, std: 0.00847, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.00252, std: 0.00309, params: {'max_depth': 6, 'min_child_weight': 3},
  mean: 0.00252, std: 0.00309, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.00749, std: 0.00606, params: {'max_depth': 8, 'min_child_weight': 1},
  mean: 0.00871, std: 0.00633, params: {'max_depth': 8, 'min_child_weight': 2},
  mean: 0.00625, std: 0.00395, params: {'max_depth': 8, 'min_child_weight': 3},
  mean: 0.00869, std: 0.00746, params: {'max_depth': 8, 'min_child_weight': 4},
  mean: 0.00873, std: 0.00492, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.00871, std: 0.00631, params: {'max_depth': 9, 'min_child_weight': 2},
  mean: 0.00745, std: 0.00463, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.01245, std: 0.00878, params: {'max_depth': 9, 'min_child_weight': 4},
  mean: 0.00743, std: 0.00910, params: {

In [11]:
param_test1 = {
 'max_depth':[11,12,13,14],
 'min_child_weight':[1,2,3,4]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=9,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X[predictors][:20000],X[target][:20000])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

([mean: 0.00374, std: 0.00500, params: {'max_depth': 11, 'min_child_weight': 1},
  mean: 0.00873, std: 0.00635, params: {'max_depth': 11, 'min_child_weight': 2},
  mean: 0.00251, std: 0.00308, params: {'max_depth': 11, 'min_child_weight': 3},
  mean: 0.00624, std: 0.00685, params: {'max_depth': 11, 'min_child_weight': 4},
  mean: 0.00746, std: 0.00725, params: {'max_depth': 12, 'min_child_weight': 1},
  mean: 0.00249, std: 0.00305, params: {'max_depth': 12, 'min_child_weight': 2},
  mean: 0.00997, std: 0.00841, params: {'max_depth': 12, 'min_child_weight': 3},
  mean: 0.00623, std: 0.00395, params: {'max_depth': 12, 'min_child_weight': 4},
  mean: 0.00502, std: 0.00731, params: {'max_depth': 13, 'min_child_weight': 1},
  mean: 0.00625, std: 0.00687, params: {'max_depth': 13, 'min_child_weight': 2},
  mean: 0.00624, std: 0.00968, params: {'max_depth': 13, 'min_child_weight': 3},
  mean: 0.00748, std: 0.00253, params: {'max_depth': 13, 'min_child_weight': 4},
  mean: 0.00498, std: 0.0061

# Tune Gamma

In [9]:
import datetime
print(datetime.datetime.now())

param_test3 = {
 'gamma': [0.4,4,8,12,16,20]#[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=6,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X[predictors],X[target])
print(datetime.datetime.now())

gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

2018-10-17 21:54:11.898409


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


2018-10-17 23:36:40.135906


([mean: 0.00466, std: 0.00075, params: {'gamma': 0.4},
  mean: 0.00441, std: 0.00098, params: {'gamma': 4},
  mean: 0.00434, std: 0.00053, params: {'gamma': 8},
  mean: 0.00378, std: 0.00074, params: {'gamma': 12},
  mean: 0.00257, std: 0.00054, params: {'gamma': 16},
  mean: 0.00089, std: 0.00053, params: {'gamma': 20}],
 {'gamma': 0.4},
 0.004656478775967863)

# Tune subsample and colsample_bytree

In [13]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=6,
 min_child_weight=6, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X[predictors],X[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

([mean: 0.00641, std: 0.00171, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.00529, std: 0.00093, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.00465, std: 0.00131, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.00489, std: 0.00093, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.00537, std: 0.00144, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.00489, std: 0.00111, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.00545, std: 0.00054, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.00529, std: 0.00114, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.00545, std: 0.00019, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.00489, std: 0.00059, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.00505, std: 0.00083, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.00425, std: 0.00069, params: {'colsample_bytree': 0.8, 'subsample'

# Tuning Regularization Parameters

In [15]:
print(datetime.datetime.now())
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=6,
 min_child_weight=6, gamma=0.4, subsample=0.6, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X[predictors],X[target])
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_
#print(datetime.datetime.now())

2018-10-17 16:28:55.881025


([mean: 0.71481, std: 0.00289, params: {'reg_alpha': 1e-05},
  mean: 0.71437, std: 0.00302, params: {'reg_alpha': 0.01},
  mean: 0.71446, std: 0.00228, params: {'reg_alpha': 0.1},
  mean: 0.71552, std: 0.00219, params: {'reg_alpha': 1},
  mean: 0.70146, std: 0.00207, params: {'reg_alpha': 100}],
 {'reg_alpha': 1},
 0.7155174327570373)

In [7]:
predictors = [x for x in X.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=9,
 min_child_weight=4, 
    gamma=0.1, 
    subsample=0.6, 
    colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 reg_alpha=1,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X[:20000], predictors)


Model Report
Accuracy : 0.9219
AUC Score (Train): 0.8815113555465445
Precision (Train): 1.0
Recall (Train): 0.009511731135066582
TN: 18423, FP: 0, FN: 1562, TP: 15


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


TypeError: 'str' object is not callable

In [8]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=9,
 min_child_weight=4, 
    gamma=0.1, 
    subsample=0.6, 
    colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 reg_alpha=1,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X, predictors)

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 0.9201446529825726
AUC Score (Train): 0.8446340043597154
Precision (Train): 0.875
Recall (Train): 0.012700076603636657
TN: 282370, FP: 45, FN: 24488, TP: 315


TypeError: 'str' object is not callable

In [16]:
#after tuning the classifier
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=6,
 min_child_weight=6, 
    gamma=0.4, 
    subsample=0.6, 
    colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 reg_alpha=1,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X, predictors)

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 0.9195652598480558
AUC Score (Train): 0.7862536534720074
Precision (Train): 0.7446808510638298
Recall (Train): 0.00564447849050518
TN: 282367, FP: 48, FN: 24663, TP: 140


# Tune scale_pos_weight

In [14]:
#after tuning the classifier
#tune scale_pos_weight
scale = (np.where(X['TARGET'] == 1)[0].shape[0])/ (np.where(X['TARGET'] == 0)[0].shape[0])
print(scale)
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=6,
 min_child_weight=6, 
    gamma=0.4, 
    subsample=0.6, 
    colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 reg_alpha=1,
 scale_pos_weight=scale,
 seed=27)
modelfit(xgb1, X, predictors)

0.08782465520599118


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 0.9192657982279684
AUC Score (Train): 0.7523693080740795
Precision (Train): 0.0


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Recall (Train): 0.0
TN: 282415, FP: 0, FN: 24803, TP: 0


In [15]:
#after tuning the classifier
#tune scale_pos_weight
scale = (np.where(X['TARGET'] == 1)[0].shape[0])/ (np.where(X['TARGET'] == 0)[0].shape[0])
print(scale)
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=6,
 min_child_weight=6, 
    gamma=0.4, 
    subsample=0.6, 
    colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 reg_alpha=1,
 scale_pos_weight=8,
 seed=27)
modelfit(xgb1, X, predictors)

0.08782465520599118


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 0.8093796587439538
AUC Score (Train): 0.7904815021419116
Precision (Train): 0.22282796105026356
Recall (Train): 0.5471112365439664
TN: 235086, FP: 47329, FN: 11233, TP: 13570


In [16]:
#after tuning the classifier
#tune scale_pos_weight
scale = (np.where(X['TARGET'] == 1)[0].shape[0])/ (np.where(X['TARGET'] == 0)[0].shape[0])
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=6,
 min_child_weight=6, 
    gamma=0.4, 
    subsample=0.6, 
    colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 reg_alpha=1,
 scale_pos_weight=9,
 seed=27)
modelfit(xgb1, X, predictors)

0.08782465520599118


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 0.7852241730627763
AUC Score (Train): 0.7977283820362966
Precision (Train): 0.2130153597413096
Recall (Train): 0.6161754626456477
TN: 225952, FP: 56463, FN: 9520, TP: 15283


In [11]:
import datetime
print(datetime.datetime.now())
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=100, gamma=16, reg_lambda=40, reg_alpha=40,n_jobs=16,scale_pos_weight=4)
modelfit(xgb1, X, predictors)
print(datetime.datetime.now())


2018-10-15 19:50:25.716157


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 0.9056240194259451
AUC Score (Train): 0.7082659406688593
Precision (Train): 0.2839022378055069
Recall (Train): 0.11099463774543401
TN: 275471, FP: 6944, FN: 22050, TP: 2753


TypeError: 'str' object is not callable

In [74]:
X2 = X
X2 = X2.drop[IDcol]
clf = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
tn, fp, fn, tp = metricsconfusion_matrix(y_test, predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

81375 3366 6250 1175


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
clf = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

81375 3366 6250 1175


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [145]:
clf = XGBClassifier(max_depth=2, min_child_weight=1, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

83443 1298 6924 501


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
clf = XGBClassifier(max_depth=12, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
clf = clf.fit(X_train[:20000], y_train[:20000])
predict = clf.predict(X_test[:20000])
tn, fp, fn, tp = confusion_matrix(y_test[:20000], predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

18067 326 1489 118


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [68]:
clf = SVC()
clf = clf.fit(X_train[:20000], y_train[:20000])
predict = clf.predict(X_test[:20000])
tn, fp, fn, tp = confusion_matrix(y_test[:20000], predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

18393 0 1607 0
